<a href="https://colab.research.google.com/github/entanglement-nak/portfolio-nak/blob/main/%E3%83%9D%E3%83%BC%E3%83%88%E3%83%95%E3%82%A9%E3%83%AA%E3%82%AA%E3%80%80xgboost%E4%BD%BF%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from xgboost import XGBClassifier

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# データパス
train_path = r"/content/drive/MyDrive/Wafer/Wafer_TRAIN.xlsx"

test_path = r"/content/drive/MyDrive/Wafer/Wafer_TEST.xlsx"

In [4]:
train_df = pd.read_excel(train_path, header=None)

test_df = pd.read_excel(test_path, header=None)

In [5]:
# トレインデータとテストデータを縦に統合（スタック）
combined_data = pd.concat([train_df, test_df], axis=0)

In [6]:
# データを特徴量(X)とターゲット(y)に分割
X = combined_data.drop(combined_data.columns[0], axis=1)
y = combined_data[combined_data.columns[0]]

In [7]:
# データを訓練用とテスト用に7:3の比率で分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
# 訓練データ内で正常と異常のクラスを分離し、異常クラスのアップサンプリングを行う
df_train = pd.concat([X_train, y_train], axis=1)
df_normal_train = df_train[df_train[df_train.columns[-1]] == 1]
df_anomaly_train = df_train[df_train[df_train.columns[-1]] == -1]

df_anomaly_upsampled = resample(df_anomaly_train, replace=True, n_samples=len(df_normal_train), random_state=123)
df_upsampled_train = pd.concat([df_normal_train, df_anomaly_upsampled])

In [9]:
# アップサンプリングされたデータを特徴量(X)とターゲット(y)に分割
X_train_upsampled = df_upsampled_train.drop(df_upsampled_train.columns[-1], axis=1)
y_train_upsampled = df_upsampled_train[df_upsampled_train.columns[-1]]

In [10]:
# ラベルの変換（-1を0に変換）
y_train_upsampled = y_train_upsampled.map({-1: 0, 1: 1})
y_test = y_test.map({-1: 0, 1: 1})

In [13]:
# XGBoost分類器の初期化
xgb = XGBClassifier()

# モデルのトレーニング
xgb.fit(X_train_upsampled, y_train_upsampled)

# テストセットに対する予測
y_pred = xgb.predict(X_test)

# モデルの評価
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(report)
# モデルの評価
accuracy = accuracy_score(y_test, y_pred)
MCC = matthews_corrcoef(y_test, y_pred)
F1 = f1_score(y_test, y_pred, average='binary')
report = classification_report(y_test, y_pred)

print("\n")
print(f"正解率：{accuracy}")
print(f"MCC：{MCC}")
print(f"F1：{F1}")

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       216
           1       1.00      1.00      1.00      1934

    accuracy                           1.00      2150
   macro avg       0.99      1.00      1.00      2150
weighted avg       1.00      1.00      1.00      2150



正解率：0.9986046511627907
MCC：0.992356503372486
F1：0.9992238033635188
